In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import pandas as pd
import sklearn

In [152]:
%store -r cook_county_gdf_quarterly
%store -r CC_nodeaths
cook_county_gdf = cook_county_gdf_quarterly

In [153]:
#add in missing 58 tracts
cook_county_gdf['deaths'] = 1
cook_county_gdf = pd.concat([cook_county_gdf, CC_nodeaths], ignore_index=True)
cook_county_gdf['quarter'].fillna(9999, inplace=True)
cook_county_gdf['year'].fillna(9999, inplace=True)

In [154]:
cook_county_gdf.loc[cook_county_gdf['year'] == 9999, 'GEOID'].nunique()

58

In [155]:
columns_to_keep = ['year', 'STATEFP', 'COUNTYFP', 'TRACTCE','NAME', 'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER',
       'INTPTLAT', 'INTPTLON', 'geometry']

gdf_quarter = cook_county_gdf.groupby(['GEOID', 'quarter']).agg({'deaths': 'sum', **{col: 'first' for col in columns_to_keep}}).reset_index()

In [156]:
gdf_quarter

,GEOID,quarter,deaths,year,STATEFP,COUNTYFP,TRACTCE,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,17031010100,1.0,5,2017.0,17,031,010100,101,Census Tract 101,G5020,S,379511,0,+42.0212553,-087.6698301,POINT (-87.66805 42.02216)
1,17031010100,2.0,6,2016.0,17,031,010100,101,Census Tract 101,G5020,S,379511,0,+42.0212553,-087.6698301,POINT (-87.67142 42.02143)
2,17031010100,3.0,7,2019.0,17,031,010100,101,Census Tract 101,G5020,S,379511,0,+42.0212553,-087.6698301,POINT (-87.67334 42.02293)
3,17031010100,4.0,6,2016.0,17,031,010100,101,Census Tract 101,G5020,S,379511,0,+42.0212553,-087.6698301,POINT (-87.66686 42.02280)
4,17031010201,1.0,4,2021.0,17,031,010201,102.01,Census Tract 102.01,G5020,S,504464,0,+42.0160077,-087.6801484,POINT (-87.68122 42.01311)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3880,17031980000,1.0,4,2019.0,17,031,980000,9800,Census Tract 9800,G5020,S,19890200,92402,+41.9794191,-087.9024376,POINT (-87.90398 41.97572)
3881,17031980000,2.0,6,2018.0,17,031,980000,9800,Census Tract 9800,G5020,S,19890200,92402,+41.9794191,-087.9024376,POINT (-87.90398 41.97572)
3882,17031980000,4.0,5,2016.0,17,031,980000,9800,Census Tract 9800,G5020,S,19890200,92402,+41.9794191,-087.9024376,POINT (-87.90647 41.97684)
3883,17031980100,4.0,2,2018.0,17,031,980100,9801,Census Tract 9801,G5020,S,2981781,0,+41.7859813,-087.7509001,POINT (-87.74137 41.78952)


### Making DF square - one quarter for each tract


In [157]:
gdf_quarter.loc[gdf_quarter['year'] == 9999, 'deaths'] = 0
gdf_quarter.loc[gdf_quarter['quarter'] == 9999, 'deaths'] = 0

# Replace 9999 with 2014 in the 'year' column
gdf_quarter['year'].replace(9999, 2014, inplace=True)
gdf_quarter['quarter'].replace(9999, 1, inplace=True)

In [158]:
gdf_quarter['GEOID'].nunique() 

1332

In [160]:
years = range(2014, 2024)
quarters = range(1, 5)
updated_rows = []

for tract in gdf_quarter['GEOID'].unique():
    existing_years_quarters = set(
        tuple(zip(gdf_quarter[gdf_quarter['GEOID'] == tract]['year'], gdf_quarter[gdf_quarter['GEOID'] == tract]['quarter'])))
    missing_years_quarters = set(
        tuple(zip(years, quarters))) - existing_years_quarters

    for year, quarter in missing_years_quarters:
        new_row = {
            'GEOID': tract,
            'year': year,
            'quarter': quarter,
            'deaths': 0
        }

        # duplicate values
        above_rows = gdf_quarter[(gdf_quarter['GEOID'] == tract) & (gdf_quarter['year'] < year)]

        if len(above_rows) > 0:
            above_rows = above_rows.sort_values(by=['year', 'quarter'])
            above_row = above_rows.tail(1)
            for col in columns_to_keep:
                new_row[col] = above_row[col].values[0]

        updated_rows.append(new_row)

gdf_quarter = pd.concat([gdf_quarter, pd.DataFrame(updated_rows)], ignore_index=True)


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
gdf_quarter['year'] = pd.to_datetime(gdf_quarter['year'].astype(int).astype(str), format='%Y').dt.year
gdf_quarter['quarter'] = gdf_quarter['quarter'].astype(str).replace('1.0', '1').replace('2.0', '2').replace('3.0', '3').replace('4.0', '4')



In [ ]:
gdf_quarter


,GEOID,quarter,deaths,year,STATEFP,COUNTYFP,TRACTCE,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
3885,17031010100,1,0,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3886,17031010100,2,0,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3887,17031010100,3,0,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3888,17031010100,4,0,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3889,17031010100,1,0,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3883,17031980100,4,2,2018,17,031,980100,9801,Census Tract 9801,G5020,S,2981781.0,0.000000e+00,+41.7859813,-087.7509001,POINT (-87.74136736446894 41.78951631196758)
3884,17031990000,1,0,2014,17,031,990000,9900,Census Tract 9900,G5020,S,0.0,1.717072e+09,+41.9739391,-087.4002947,"POLYGON ((-87.741562 42.153226, -87.721762 42...."
53241,17031990000,2,0,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53242,17031990000,3,0,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
gdf_quarter.shape #should have 53280 rows

(53280, 16)

In [ ]:
gdf_quarter['GEOID'].nunique()

1332

In [ ]:
years = range(2014, 2024)
quarters = range(1, 5)

for tract in gdf_quarter['GEOID'].unique():
    for year in years:
        quarters_present = set()
        for quarter in quarters:
            if (tract, year, quarter) in tuple(zip(gdf_quarter['GEOID'], gdf_quarter['year'], gdf_quarter['quarter'])):
                quarters_present.add(quarter)

        if len(quarters_present) != 4:
            print(f"Missing quarters for GEOID {tract} in year {year}")


Missing quarters for GEOID 17031010100 in year 2014
Missing quarters for GEOID 17031010100 in year 2015
Missing quarters for GEOID 17031010100 in year 2016
Missing quarters for GEOID 17031010100 in year 2017
Missing quarters for GEOID 17031010100 in year 2018
Missing quarters for GEOID 17031010100 in year 2019
Missing quarters for GEOID 17031010100 in year 2020
Missing quarters for GEOID 17031010100 in year 2021
Missing quarters for GEOID 17031010100 in year 2022
Missing quarters for GEOID 17031010100 in year 2023
Missing quarters for GEOID 17031010201 in year 2014
Missing quarters for GEOID 17031010201 in year 2015
Missing quarters for GEOID 17031010201 in year 2016
Missing quarters for GEOID 17031010201 in year 2017
Missing quarters for GEOID 17031010201 in year 2018
Missing quarters for GEOID 17031010201 in year 2019
Missing quarters for GEOID 17031010201 in year 2020
Missing quarters for GEOID 17031010201 in year 2021
Missing quarters for GEOID 17031010201 in year 2022
Missing quar

KeyboardInterrupt: 

### Tables and Figures